In [ ]:
from db.models import Tag

# query logic for retrieving tagged file embedding for use in test.
DEFAULT_CHILD_TAG_N = 50

# Get the child tags for set of parent tag targets.
target_tag_list = ['B', 'C', 'D', 'E', 'F', 'G', 'H']
target_tag_query = Tag.select().where(Tag.name.in_(target_tag_list))

